In [22]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset, Dataset
import os
import pandas as pd
from tqdm.notebook import tqdm
import time
import gc
import re
import numpy as np

In [2]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
model_id = "openai/whisper-large-v3"
print(f'{device = } | {torch_dtype = }')

device = 'cuda:0' | torch_dtype = torch.float16


In [3]:
model = AutoModelForSpeechSeq2Seq.from_pretrained(model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True,
                                                 device_map="cuda").eval()
processor = AutoProcessor.from_pretrained(model_id)

In [4]:
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=400,
    chunk_length_s=30,  # or Sequential or chunking
    batch_size=12,
    return_timestamps=True,
    torch_dtype=torch_dtype,
)

In [5]:
def reorder_text(df, time_step=120):
    new = []
    cur_start_time = 0
    cur_max_time = time_step
    cur_text = []
    for i in range(len(df)):
        end = df.loc[i, 'end']
        text = df.loc[i, 'text']
        if end < cur_max_time:
            cur_text.append(text.strip())
        else:
            text_chunk = ' '.join(cur_text).replace('  ', ' ').strip()
            if len(text_chunk) > 4:
                new.append([cur_start_time, df.loc[i, 'start'], ])
            cur_start_time = df.loc[i, 'start']
            cur_max_time += time_step
            cur_text = [text.strip()]
    new.append([cur_start_time, end, ' '.join(cur_text).replace('  ', ' ')])
    return pd.DataFrame(new, columns=['start', 'end', 'text'])

In [6]:
with open(os.path.join('../tmp_work_files', 'audio_files', 'processed_files.txt'), 'r') as f:
    processed_files = [p.replace('\n', '') for p in f.readlines()]
    
def update_processed(filename):
    global processed_files
    processed_files.append(filename)
    with open(os.path.join('../tmp_work_files/audio_files', 'processed_files.txt'), 'a') as f:
        f.writelines(processed_files[-1] + '\n')

len(processed_files)

694

# ASR from folder audio files

In [ ]:
files = tqdm([o for o in os.listdir('../tmp_work_files/audio_files/') if o.endswith('m4a')][660:])
for filename in files:
    youtube_link = re.search(r"(\[[^\]]+\]).m4a$", filename).group(1)
    if youtube_link in processed_files:
        continue
        
    files.set_description(filename[:40])
    with torch.no_grad():
        result = pipe(os.path.join("../tmp_work_files/audio_files", filename), generate_kwargs={"num_beams": 5})
    gc.collect()
    torch.cuda.empty_cache()

    # result['chunks'] = fix_timestamps(result['chunks'])  #  !!!! for sequential
    
    ds = pd.DataFrame()    
    ds['start'] = [r['timestamp'][0] for r in result['chunks']]
    ds['end'] = [r['timestamp'][1] for r in result['chunks']]
    ds['text'] = [r['text'] for r in result['chunks']]
    
    ds = reorder_text(ds)
    ds.to_csv(os.path.join("../tmp_work_files/texts_from_audio", os.path.splitext(filename)[0] + ".csv"), index=False)
    update_processed(youtube_link)

In [7]:
reorder_text(ds)

,start,end,text
0,0.00,103.30,вот ну мы решили сегодня проверить по аллахе к...
1,103.30,238.38,"теперь у него возникает сомнение, возможно, он..."
2,238.38,356.38,весь и давать если внизу у сима отд Мишнабрура...
3,356.38,474.36,читают указы Верховного Совета все время. Прав...
4,474.36,576.20,"Ки адам, а курэ цивуэ амерах, шо курэ бэ метун..."
5,576.20,713.06,это получилось а 248 кенеги дайвара в Тайвот и...
6,713.06,843.12,"Достаточно, что ты слышишь, тебя как бы освобо..."
7,843.12,947.00,"Тибур, смесаем, абраха, векен, нуагим, винахон..."
8,947.00,1077.00,поэтому конечно не хочу путать всех разные нос...
9,1077.00,1198.52,"Вы добавляете «зайн», да? «Зайн». Видите, «зай..."


---

In [6]:
ds = pd.read_csv(r"D:\Programming\My_reps\MM_telegram_bot\tmp_dev\tmp\Тайны загробного мира. Талмуд по-русски. Рав Йосеф Менделевич #09-10-2024# [PQBXwFdsIIE].csv")

In [13]:
ds.loc[:len(ds) - 2, 'text'] = ' '

In [25]:
ds['text'] = ds['text'].apply(lambda x: x.strip() if len(x.strip()) > 4 else np.nan)

In [21]:
len(ds.loc[0, 'text'])

0

In [30]:
ds = ds[ds['text'].notna()].reset_index()

In [31]:
ds

,index,start,end,text
0,33,3960.36,3994.12,но не все время будем учить Гимаре про мертвых...


In [29]:
reorder_text(ds)

KeyError: 0

# test

In [10]:
file_path = os.listdir('audio_files/')[0]
file_path

'В жизни логика не работает. Урок №4. Талмуд по-русски. Рав Йосеф Менделевич [S5zLqJ2JSnE].m4a'

Sequential: 8 mins 57 sec  
Chunked: 3 mins 45 sec

In [22]:
st_time = time.perf_counter()
with torch.no_grad():
    result = pipe("../tmp_work_files/rav.m4a", generate_kwargs={"language": "russian", "num_beams": 5, "best_of": 5})
gc.collect()
torch.cuda.empty_cache()
tt = time.perf_counter() - st_time
print(f"{tt // 60:.0f} mins {tt % 60:.0f} sec")

C:\Users\Otniel\.conda\envs\p310\lib\site-packages\transformers\models\whisper\generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


3 mins 40 sec


In [12]:
result.keys()

dict_keys(['text', 'chunks'])

In [17]:
def fix_timestamps(chunks):
    new_ts = []
    last_end = 0
    additional_time = 0
    for ch in chunks:
        if ch['timestamp'][0] == 0:
            additional_time += last_end
        new_ts.append({'timestamp': (ch['timestamp'][0] + additional_time, ch['timestamp'][1] + additional_time),
                      "text": ch['text']})
        
        last_end = ch['timestamp'][1]
    return new_ts 

In [18]:
fix_timestamps(result['chunks'])

[{'timestamp': (0.0, 12.64),
  'text': ' С врагом надо уметь не только быть жестоким, но и уметь и хитрить.'},
 {'timestamp': (14.0, 21.04),
  'text': ' Против нас работает опасный враг, и его жестокость никого не удивляет.'},
 {'timestamp': (22.04, 26.86),
  'text': ' А вот то, что он хитрый и наглый, а мы, евреи,'},
 {'timestamp': (26.86, 37.0),
  'text': ' даже какие не были бы самые серьезные, есть какой-то элемент наивности.'},
 {'timestamp': (37.2, 40.08),
  'text': ' Надеешься, ну не может быть, чтобы человек так поступил.'},
 {'timestamp': (40.5, 43.4), 'text': ' Так вот, на этот раз это не проходит.'},
 {'timestamp': (43.9, 47.08), 'text': ' Нужно быть наглым до конца.'},
 {'timestamp': (47.08, 65.02),
  'text': ' О чем я говорю? Сейчас у нас на горизонте нарисовывается соглашение о прекращении огня временно для того, чтобы освободить схваченных невинных людей.'},
 {'timestamp': (65.9, 73.1),
  'text': ' Называется заложники, ну может быть и заложники, может называть, хатуфим.

In [24]:
# chunking
result['chunks']

[{'timestamp': (0.0, 12.64),
  'text': ' С врагом надо уметь не только быть жестоким, но и уметь и хитрить.'},
 {'timestamp': (14.0, 21.04),
  'text': ' Против нас работает опасный враг, и его жестокость никого не удивляет.'},
 {'timestamp': (22.04, 32.0),
  'text': ' А вот то, что он хитрый и наглый а мы евреи даже какие не были бы самые серьезные'},
 {'timestamp': (32.0, 65.02),
  'text': ' есть какая-то какой-то элемент наивности надеюсь ему не может быть чтобы человек так поступил так Так вот, на этот раз это не проходит. Нужно быть наглым до конца. О чем я говорю? Сейчас у нас на горизонте нарисовывается соглашение о прекращении огня временным для того, чтобы освободить схваченных невинных людей.'},
 {'timestamp': (65.88, 68.88),
  'text': ' Называются заложники. Ну, может быть, и заложники можно называть.'},
 {'timestamp': (68.94, 73.08),
  'text': ' Хатуфим. Хатуфим – это схваченные, схваченные люди.'},
 {'timestamp': (74.02, 80.08),
  'text': ' И вот, значит, есть какие-то усло